In [1]:
import pandas as pd
path = 'Minhas Listas - Filmes.csv'

In [ ]:
my = pd.read_csv(path, low_memory=False, index_col='tconst' )
basics = pd.read_csv('imdb_dataset/title.basics.tsv', sep='\t', low_memory=False, index_col='tconst')
cast = pd.read_csv('imdb_dataset/title.principals.tsv', sep='\t', low_memory=False, index_col='tconst')
crew = pd.read_csv('imdb_dataset/title.crew.tsv', sep='\t', low_memory=False, index_col='tconst')
names = pd.read_csv('imdb_dataset/name.basics.tsv', sep='\t', low_memory=False)
ratings = pd.read_csv('imdb_dataset/title.ratings.tsv', sep='\t', low_memory=False, index_col='tconst') 

In [ ]:
new_entry = 'tt0085995'
entry_dict = {'tconst':new_entry}

In [ ]:
entry_dict['originalTitle'] = basics.loc[new_entry, 'originalTitle']
entry_dict['startYear'] = int(basics.loc[new_entry, 'startYear'])
entry_dict['runtimeMinutes'] = int(basics.loc[new_entry, 'runtimeMinutes'])
entry_dict['genres'] = basics.loc[new_entry, 'genres']
entry_dict['directors'] = crew.loc[new_entry, 'directors']
entry_dict['writers'] = crew.loc[new_entry, 'writers']
entry_dict['averageRating'] = float(ratings.loc[new_entry, 'averageRating'])
entry_dict['numVotes'] = int(ratings.loc[new_entry, 'numVotes'])
cast = cast.loc[(cast['category'] == 'actor') | (cast['category'] == 'actress')]
result = cast.loc[new_entry].groupby('tconst')['nconst'].apply(lambda x: ', '.join(x)).reset_index()
entry_dict['nconst'] = result['nconst'].iloc[0]


code_to_name = dict(zip(names['nconst'], names['primaryName']))

def replace_codes_with_names(cell):
    if pd.isna(cell):
        return cell
    parts = [p.strip() for p in str(cell).split(',')]
    replaced = []
    for part in parts:
        if part == '':
            continue
        nome = code_to_name.get(part, part)
        replaced.append(nome)
    return ', '.join(replaced)

entry_dict['directors'] = replace_codes_with_names(entry_dict['directors'])
entry_dict['writers'] = replace_codes_with_names(entry_dict['writers'])
entry_dict['nconst'] = replace_codes_with_names(entry_dict['nconst'])

In [7]:
entry_dict

{'tconst': 'tt0085995',
 'originalTitle': "National Lampoon's Vacation",
 'startYear': 1983,
 'runtimeMinutes': 98,
 'genres': 'Adventure,Comedy',
 'directors': 'Harold Ramis',
 'writers': 'John Hughes',
 'averageRating': 7.3,
 'numVotes': 124678,
 'nconst': "Chevy Chase, Beverly D'Angelo, Imogene Coca, Randy Quaid, Anthony Michael Hall, Dana Barron, Eddie Bracken, Brian Doyle-Murray, Miriam Flynn, James Keach"}

In [8]:
entry_dict['personal_score'] = 6


In [9]:
new_entry_df = pd.DataFrame([entry_dict])
new_entry_df.set_index('tconst', inplace=True)
my = pd.concat([my, new_entry_df], axis=0)


In [20]:
my.loc[new_entry]

originalTitle                           National Lampoon's Vacation
personal_score                                                  6.0
averageRating                                                   7.3
startYear                                                      1983
genres                                             Adventure,Comedy
runtimeMinutes                                                   98
directors                                              Harold Ramis
writers                                                 John Hughes
numVotes                                                     124678
nconst            Chevy Chase, Beverly D'Angelo, Imogene Coca, R...
Name: tt0085995, dtype: object

In [19]:
my.drop_duplicates(inplace=True)
my.to_csv(path)